In [1]:
from data.generator import *

2024-03-07 13:24:37.293684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
n = 6
p = 0.3

dataset = RandomGraphDataset(root='./data', gen_num_graph=10, n=n, p=p)

Generating 10 graphs


100%|██████████| 10/10 [00:00<00:00, 1270.46it/s]
Processing...
100%|██████████| 10/10 [00:00<00:00, 437.01it/s]
Done!


In [3]:
import torch
from torch_geometric.nn import MessagePassing
from torch.nn import Linear

class MPNN(MessagePassing):
  def __init__(self, in_channels, hidden_channels, activation=None):
    super(MPNN, self).__init__(aggr='max') #  "Max" aggregation.
    self.in_channels = in_channels
    self.hidden_channels = hidden_channels
    self.messages = Linear(self.in_channels * 2, self.hidden_channels)
    self.update_fn = Linear(self.in_channels + self.hidden_channels, self.hidden_channels)
    self.activation = activation

    self.mlp = torch.nn.Sequential(
        Linear(in_channels, hidden_channels),
        torch.nn.ReLU(),
        Linear(hidden_channels, hidden_channels)
    )
    
  def forward(self, x, edge_index):
    out = self.propagate(edge_index, x=x)
    out = self.mlp(out)
    if self.activation is not None:
      out = self.activation(out)
    return out
    
  def message(self, x_i, x_j):
    # x_i has shape [E, in_channels]
    # x_j has shape [E, in_channels]
    #print('MPNN => xi, xj', x_i.size(), x_j.size())
    tmp = torch.cat([x_i, x_j], dim=1)  # tmp has shape [E, 2 * in_channels]
    #print('MPNN => messages IN', tmp.size())
    m = self.messages(tmp)
    #print('MPNN => messages OUT', m.size())
    return m
  
  def update(self, aggr_out, x):
    # aggr_out has shape [N, out_channels]
    # x has shape [N, in_channels]
    #print(f'MPNN => x_i', x.size(), ' aggr_out ', aggr_out.size())
    tmp = torch.cat([x, aggr_out], dim=1)
    #print(f'MPNN => tmp', tmp.size())
    return self.update_fn(tmp)

In [4]:
# create an encoder class
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.lin = nn.Linear(1, hidden_dim)

    def forward(self, x):
        if x.dim() == 1:
            x = x.unsqueeze(-1)
        return self.lin(x)

In [5]:
from torch import nn
class Decoder(nn.Module):
    def __init__(self, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lin = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        return self.lin(x)

In [6]:
from torch.functional import F
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()

    def forward(self, x: torch.Tensor, x_pred: torch.Tensor, h: torch.Tensor, h_pred: torch.Tensor):
        """
        x: true output value
        x_pred: predicted output value
        h: true hint values
        h_pred: predicted hint values
        """

        loss_x = F.binary_cross_entropy(x, x_pred) # binary cross entropy loss between true and predicted output
        hints_loss = 0
        for i in range(h.size(1)):
            hints_loss += F.binary_cross_entropy(h[:, i], h_pred[:, i])

        return loss_x + hints_loss

In [7]:
from torch.functional import F
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()

    def forward(self, batch, batch_pred: torch.Tensor):

        # for every batch find the predicted and true values and send them to the calculate_loss function
        loss_x = 0
        loss_h = 0
        for i in range(batch.len()):
            data = batch[i]
            x = data.reach_h[-1] # true output value
            x_pred = batch_pred[i][-1] # predicted output value
            h_pred = batch_pred[i][:-2] # predicted hint values
            h = data.reach_h[:len(h_pred)] # true hint values
            print(x)
            print(x_pred)
            loss_x += F.binary_cross_entropy(x, x_pred)
            print(loss_x)
            for i in range(h.size(1)):
                loss_h += F.binary_cross_entropy(h[:, i], h_pred[:, i])

        return loss_x + loss_h

    
    # def calculate_loss(self, x, x_pred, h, h_pred):
    #     """
    #     x: true output value
    #     x_pred: predicted output value
    #     h: true hint values
    #     h_pred: predicted hint values
    #     """
    #     loss_x = F.binary_cross_entropy(x, x_pred) # binary cross entropy loss between true and predicted output
    #     hints_loss = 0
    #     for i in range(h.size(1)):
    #         hints_loss += F.binary_cross_entropy(h[:, i], h_pred[:, i])

    #     return loss_x + hints_loss

In [22]:
predictions = []
for i in np.arange(dataset.len()):
    data = dataset[i]
    predictions.append(data.reach_h)

In [ ]:
predictions

In [19]:
predictions = []
for d in dataset:
    h_pred = []
    # generate a prediction wich is the same size as the reach_h
    len = d.reach_h[-1].size(0)
    hints_len = d.reach_h.size(1)
    # create as many predictions as there are hints
    for i in range(hints_len):
        h_pred.append(torch.rand(len))
    # append the prediction to the list
    predictions.append(h_pred)
predictions

[[tensor([0.7143, 0.1741, 0.1920, 0.8777, 0.7658, 0.3378]),
  tensor([0.9543, 0.3689, 0.3727, 0.8353, 0.5775, 0.6603]),
  tensor([0.2558, 0.1154, 0.5615, 0.4870, 0.9528, 0.1437]),
  tensor([0.3564, 0.6232, 0.4392, 0.9562, 0.7816, 0.5164]),
  tensor([0.1900, 0.4127, 0.0310, 0.2288, 0.3915, 0.8450]),
  tensor([0.5902, 0.5273, 0.9557, 0.9079, 0.9055, 0.9465])],
 [tensor([0.9952, 0.3400, 0.4219, 0.8635, 0.0802, 0.3195]),
  tensor([0.1953, 0.2896, 0.1495, 0.2523, 0.5475, 0.4005]),
  tensor([0.9543, 0.6819, 0.0392, 0.5422, 0.2216, 0.6485]),
  tensor([0.7830, 0.5166, 0.4493, 0.7869, 0.9324, 0.0452]),
  tensor([0.0222, 0.3534, 0.2351, 0.8061, 0.3082, 0.0164]),
  tensor([0.5317, 0.8365, 0.4253, 0.3484, 0.1533, 0.9588])],
 [tensor([0.2869, 0.0196, 0.1265, 0.4460, 0.2904, 0.4743]),
  tensor([4.9248e-01, 3.2627e-01, 3.0349e-01, 6.0767e-04, 6.8539e-01, 5.2193e-01]),
  tensor([0.1412, 0.8722, 0.7867, 0.8790, 0.2798, 0.1149]),
  tensor([0.4754, 0.3637, 0.7234, 0.7863, 0.5685, 0.8254]),
  tensor([0.00

In [29]:
preditions = []

for d in dataset:
    predictions.append(d.reach_h)

In [24]:
predictions

[[tensor([0.7143, 0.1741, 0.1920, 0.8777, 0.7658, 0.3378]),
  tensor([0.9543, 0.3689, 0.3727, 0.8353, 0.5775, 0.6603]),
  tensor([0.2558, 0.1154, 0.5615, 0.4870, 0.9528, 0.1437]),
  tensor([0.3564, 0.6232, 0.4392, 0.9562, 0.7816, 0.5164]),
  tensor([0.1900, 0.4127, 0.0310, 0.2288, 0.3915, 0.8450]),
  tensor([0.5902, 0.5273, 0.9557, 0.9079, 0.9055, 0.9465])],
 [tensor([0.9952, 0.3400, 0.4219, 0.8635, 0.0802, 0.3195]),
  tensor([0.1953, 0.2896, 0.1495, 0.2523, 0.5475, 0.4005]),
  tensor([0.9543, 0.6819, 0.0392, 0.5422, 0.2216, 0.6485]),
  tensor([0.7830, 0.5166, 0.4493, 0.7869, 0.9324, 0.0452]),
  tensor([0.0222, 0.3534, 0.2351, 0.8061, 0.3082, 0.0164]),
  tensor([0.5317, 0.8365, 0.4253, 0.3484, 0.1533, 0.9588])],
 [tensor([0.2869, 0.0196, 0.1265, 0.4460, 0.2904, 0.4743]),
  tensor([4.9248e-01, 3.2627e-01, 3.0349e-01, 6.0767e-04, 6.8539e-01, 5.2193e-01]),
  tensor([0.1412, 0.8722, 0.7867, 0.8790, 0.2798, 0.1149]),
  tensor([0.4754, 0.3637, 0.7234, 0.7863, 0.5685, 0.8254]),
  tensor([0.00

In [24]:
# test loss on 
batch = dataset
batch_pred = predictions

loss = Loss()
loss(batch, batch_pred)

tensor([1., 0., 1., 1., 1., 0.], dtype=torch.float64)
tensor([1., 0., 1., 1., 1., 0.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 1., 0., 1., 1., 1.], dtype=torch.float64)
tensor([1., 1., 0., 1., 1., 1.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 0., 1., 0., 1., 1.], dtype=torch.float64)
tensor([1., 0., 1., 0., 1., 1.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 0., 0., 0., 0., 0.], dtype=torch.float64)
tensor([1., 0., 0., 0., 0., 0.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 1., 1., 1., 1., 0.], dtype=torch.float64)
tensor([1., 1., 1., 1., 1., 0.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 0., 0., 0., 1., 0.], dtype=torch.float64)
tensor([1., 0., 0., 0., 1., 0.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 1., 1., 1., 1., 1.], dtype=torch.float64)
tensor([1., 1., 1., 1., 1., 1.], dtype=torch.float64)
tensor(0., dtype=torch.float64)
tensor([1., 1., 1., 

tensor(nan, dtype=torch.float64)

In [40]:
d1.reach_h[:-2]

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.]], dtype=torch.float64)

In [43]:
x = d1['reach_h'][-1]
x_pred = torch.rand_like(x)
h = d1.reach_h[:-2]
h_pred = torch.rand_like(h)

In [46]:
loss = Loss()
loss(x, x_pred, h, h_pred)

tensor(13.5479, dtype=torch.float64)

In [157]:
g = nx.read_edgelist("./data/raw/er_graph_0.edgelist")
# create the adjacency matrix knowing that there are n nodes and n = 6
n=6

adj

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 0., 0., 0.]])

In [ ]:
g = nx.read_edgelist("./data/raw/er_graph_0.edgelist")
edges_indexes = self.get_edges_indexes(adj)
s = np.random.randint(0, len(adj))
pi, probes = bfs(adj, s)
pi_h = probes['hint']['node']['pi_h']['data']
reach_h = probes['hint']['node']['reach_h']['data']
pi = self.get_edges(adj, pi)
pi_h = np.array([self.get_edges(adj, x) for x in pi_h])
pos = np.arange(0, len(adj))/len(adj)
length = (pi_h).shape[0]
dict = {'edge_index': edges_indexes, 'pos': pos, 'length': length, 's': s, 'pi': pi, 'reach_h': reach_h, 'pi_h': pi_h}
dict = {k: self.to_torch(v) for k,v in dict.items()}
dict['hints'] = np.array(['reach_h', 'pi_h'])
dict['inputs'] = np.array(['pos', 's'])
dict['outputs'] = np.array(['pi'])
tensor = CLRSData(**dict)